<a id='problem_statement'></a>
## Problem Statements & Scope

 Problem Statement: Make dataset for ML project(s) end-to-end.

This notebook only covers the scraping and transforming of the base dataset. Each project goal will be having other data preprocessing relevant to its problem statement. The dataset is only for educational purposes, so I anonymized the user name feature and dropped the user link feature in the User Dataset. However, there's no private information in the dataset as the user profile was public and can be seen on the original site. Despite the anonymization, there's still be a snippet of some of the true user names and links in this notebook. 

#### import required modules

In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
#scraping
from urllib.request import urlopen as ureq
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from dateutil import parser
import datetime
import time

# timeout issues
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.page_load_strategy = 'normal'
chrome_options.add_argument('--enable-automation')
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-insecure-localhost')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-browser-side-navigation')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome(ChromeDriverManager().install())

#dataframes
import pandas as pd
import numpy as np
import re

#anonymize usernames
import csv
from faker import Factory
from collections import defaultdict

C:\Users\User\AppData\Local\Temp\ipykernel_15992\294728646.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
pd.set_option('display.max_columns', None)

<a id='scraping_brand'></a>
### Scraping the local brands

In [3]:
PATH = r"D:\Downloads\DataScrape\chromedriver.exe"
driver = webdriver.Chrome(PATH)

brand_url ='https://reviews.femaledaily.com/brands'
#opening up connection, grabbing the page with selenium webdriver
driver.get(brand_url)

C:\Users\User\AppData\Local\Temp\ipykernel_15992\540071313.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [4]:
#Indonesian brands
brand_idn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='id_tab_Indonesia']")))
driver.execute_script("arguments[0].click();", brand_idn)

#brands alphabet
brand_list = brand_idn.find_element(By.XPATH, "//div[@class='jsx-2959408550 brand-alphabet-list']")
brand_char = brand_list.find_elements(By.TAG_NAME, "a")

In [5]:
list_brand_lokal = []
sum_prod_lokal =[]
list_link = []

# list of local beauty brands
for char in brand_char:
	driver.execute_script("arguments[0].click();", char)
	brand_list_prod = char.find_element(By.XPATH, "//div[@class='jsx-2959408550 brand-list-prod']")
	brand_prod = brand_list_prod.find_elements(By.TAG_NAME, 'a')
	for name in brand_prod:
		brand_name = name.find_elements(By.TAG_NAME, 'h2')
		sum_prod = name.find_elements(By.TAG_NAME, 'p')
		list_link.append(name.get_attribute('href'))
		for only_name in brand_name:
			list_brand_lokal.append(only_name.text)
		for only_sum in sum_prod:
			sum_prod_lokal.append(only_sum.text)

In [6]:
len(list_brand_lokal)

1272

In [7]:
zip_iterator = zip(list_brand_lokal, list_link)
brand_dict = dict(zip_iterator)
dict_skincare_lokal = {}
link_product = []

In [ ]:
#get list of local skincare brand & products href
for key, value in brand_dict.items():
	driver.get(value)
	try:
		skincare_filter = driver.find_element(By.XPATH, "//*[contains(text(), 'SkinCare')]")
		driver.execute_script("arguments[0].click();", skincare_filter)
		element_exist = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='jsx-4252524774 brand-product']")))
		if element_exist:
			dict_skincare_lokal[key] = value
			brand_product = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='jsx-2059197805']")))
			for product in brand_product:
				link_product.append(product.get_attribute('href'))
	except:
		print("merk " + key + " tidak punya produk skincare")

merk AIMR tidak punya produk skincare
merk ARRA Beauty tidak punya produk skincare
merk AZZURA tidak punya produk skincare
merk Absolute tidak punya produk skincare
merk Aelin Skincare tidak punya produk skincare
merk Aeluna tidak punya produk skincare
merk Aeris Beaute tidak punya produk skincare
merk Aeuko tidak punya produk skincare
merk Akiko Beauty tidak punya produk skincare
merk Alatté tidak punya produk skincare
merk Alchemist Fragrance tidak punya produk skincare
merk Aldo Akira tidak punya produk skincare
merk Alien Objects tidak punya produk skincare
merk All Is Well tidak punya produk skincare
merk Allglows tidak punya produk skincare
merk Alraiya tidak punya produk skincare
merk Alurin Beauty tidak punya produk skincare
merk Amara tidak punya produk skincare
merk Amara Kids Cosmetics tidak punya produk skincare
merk Amora tidak punya produk skincare
merk Anava'S Secret tidak punya produk skincare
merk Andalan Feminine Care tidak punya produk skincare
merk Anggie Rassly Bea

In [ ]:
print(len(dict_skincare_lokal))
print(len(link_product))

<a id='scraping_product'></a>
### Scraping the local skincare products

In [ ]:
subcategory = []
product_brand = []
product_name = []
product_variant = []
product_price =[]
product_rate=[]
product_desc = []
product_link = []
product_image = []

#get information for each product
for link in link_product:
        
	driver.get(link)
	driver.implicitly_wait(3) #wait 3 seconds

	try:
		breadcrumb_sub = driver.find_elements(By.XPATH, "//a[@class='jsx-1858436673 breadcrumb-section-text']")
		get_product_brand = driver.find_element(By.XPATH, "//h2[@class='jsx-2016320139 jsx-2462230538 product-brand']")
		get_product_name = driver.find_element(By.XPATH, "//h1[@class='jsx-2016320139 jsx-2462230538 product-name']")
        # jsx-905139883 jsx-1996910461 product-name
		get_desc_toggle = driver.find_element(By.XPATH, '//span[@class="jsx-2016320139 jsx-2462230538 toggle-desc expanded"]')
		driver.execute_script("arguments[0].click();", get_desc_toggle)
		get_product_desc = driver.find_element(By.XPATH, "//p[@class='jsx-2016320139 jsx-2462230538 product-desc']")
		get_product_rate = driver.find_element(By.XPATH, "//p[@class='jsx-2016320139 jsx-2462230538 ']")
		get_product_image = driver.find_element(By.XPATH, "//div[@class='jsx-2016320139 jsx-2462230538 image-wrapper']//img")
# 		get_product_image = get_product_image.getAttribute("get_product_image")
   
		#get product price
		try:
			get_product_price = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, "//p[@class='jsx-2016320139 jsx-2462230538 product-price']")))
			if get_product_price:
				product_price.append(get_product_price[0].text)
			else:
				product_price.append('')
		except:
			product_price.append('beauty studio')
                
		#get product variant
		get_product_variant = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, '//h3[@class="jsx-2016320139 jsx-2462230538 product-shade"]')))
		if get_product_variant:
			product_variant.append(get_product_variant[0].text)
		else:
			product_variant.append('')
	        
		#append the information to lists
		subcategory.append(breadcrumb_sub[1].text)
		product_brand.append(get_product_brand.text)
		product_name.append(get_product_name.text)
		product_desc.append(get_product_desc.text)
		product_rate.append(get_product_rate.text)
		product_link.append(link)
		product_image.append(get_product_image.get_attribute('src'))
        
	except:
		print('something not found')

In [ ]:
print(len(subcategory))
print(len(product_brand))
print(len(product_name))
print(len(product_rate))
print(len(product_price))
print(len(product_desc))
print(len(product_variant))
print(len(product_image))

In [ ]:
# #anonymize usernames
import csv
from faker import Factory
from collections import defaultdict

In [ ]:
product_df = pd.DataFrame(list(zip(subcategory, product_brand, product_name, product_rate, product_variant, product_price, product_desc, product_link, product_image)), 
                  columns= ['subcategory', 'brand', 'product_name','rate', 'variant', 'price', 'description', 'link', 'image'])

In [ ]:
product_df.to_csv("skincare_product.csv", index=False, header=True)

##### [back to Table of Contents](#table_content)